In [1]:
import os
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from threading import Thread
from PIL import Image

In [2]:
import numpy as np

In [3]:
def create_path(uri):
    if not os.path.isdir(uri):
        uri_ = os.path.split(uri)[0]
        create_path(uri_)

        os.mkdir(uri)

**Definição da classe pra gerar imagens com ImageDataGenerator**

In [4]:
#Esta implementação gera novas imagens a partir das originais.
class GenerateImage(Thread):
    
    def __init__ (self, path, to, qtd):
        Thread.__init__(self)
        self.path = path
        self.to = to
        self.qtd = qtd
        
    def run(self):
        self.generate()
        
    def create_path(self,uri):
        if not os.path.isdir(uri):
            uri_ = os.path.split(uri)[0]
            self.create_path(uri_)

            os.mkdir(uri)
            

    def generate(self):
        
        self.create_path(self.to)
        
        names_images = os.listdir(self.path) #lista de nomes de arquivos que existem na pasta path
        
        #objeto de ImageDataGenerator, que gera imagens
        datagenerator = ImageDataGenerator(
            rotation_range = 90,
            #width_shift_range=0.1,
            #height_shift_range=0.1,
            rescale=1./255,
            #shear_range=0.2,
            zoom_range=0.23,
            horizontal_flip=True,
            vertical_flip=True,
            fill_mode='nearest'
        )

        #calculo da quantidade a ser repetida para cada imagem encontrada
        #count = int(self.qtd/len(names_images))
        #count = count if (count > 0) else 1
        
        print("\n\n")
        #criar imagens a partir da lista na pasta
        for name_image in names_images:
            
            print(name_image, end="\r")
            
            if not os.path.isfile(self.path+"/"+name_image): #Se não for arquivo, pula esse
                continue

            image = load_img(self.path+"/"+name_image) # carrega a imagem
            image_array = img_to_array(image)
            image_array = image_array.reshape((1,)+image_array.shape) #redimensiona a imagem

            i = 0
            for batch in datagenerator.flow(image_array, batch_size=1, save_to_dir=self.to, save_prefix="g", save_format='jpeg'):
              
              i = i+1
              if (i >= self.qtd):
                break


**redimensionar as imagens para Uma resolução desejada**

In [2]:
#redimensiona cada imagem da path para o tamanho correspondente
class CropScalingImage(Thread):
    
    def __init__ (self, path, to_path, r, with_crop, with_resize):
        Thread.__init__(self)
        self.path = path
        self.to_path = to_path
        self.r = r
        self.with_crop = with_crop
        self.with_resize = with_resize
        
    def run(self):
        self.scaling()
        
    def create_path(self,uri):
        if not os.path.isdir(uri):
            uri_ = os.path.split(uri)[0]
            self.create_path(uri_)

            os.mkdir(uri)
        
    def scaling(self):
        
        self.create_path(self.to_path)
        
        for name in os.listdir(self.path):
            
            img = Image.open(self.path+"/"+name)

            #crop image
            
            w = img.size[0]
            h = img.size[1]
            
            if self.with_crop:
                if w > h:
                    dw = (w-h)/2.0
                    img = img.crop((dw,0,w-dw,h))
                elif h > w:
                    dh = (h-w)/2.0
                    img = img.crop((0,dh,w,h-dh))
                else:
                    dw = (w-self.r)/2.0
                    dh = (h-self.r)/2.0
                    
                    rr = (dw,dh,self.r+dw,self.r+dh)
                    img = img.crop(rr)
            
            if self.with_resize:
                #scale image
                if w > h:
                    img = img.resize((self.r,self.r), Image.ANTIALIAS)
                else:
                    img = img.resize((self.r,self.r), Image.ANTIALIAS)
                    
            if not os.path.isdir(self.to_path):
                os.mkdir(self.to_path)
                
            img.save(self.to_path+"/"+name)
            

# Executar

*Apague coloque o link da pasta que contém as imagens entre aspas*

In [5]:
uri_path = "C:/Users/Joelp/OneDrive/Imagens/tireoide/citologia"
uri_to_path = "C:/Users/Joelp/OneDrive/Imagens/tireoide/generated"

**Recorte e redimensionamento**

In [5]:
import glob

In [19]:
#recortar apenas um lado.
for p in os.listdir(uri_path):
    
    for file in glob.glob(os.path.join(uri_path,p+"/*.jpg")):
        
        name = os.path.split(file)[1]
        
        image = Image.open(file)
        w,h = image.size
        image = image.crop((0,0,w-150,h))
        
        to = os.path.join(uri_to_path,p)
        create_path(to)
        image.save(os.path.join(to, name))
        

**Converter imagens png para jpg**

In [21]:
def png_to_jpg(image_uri):
        png = Image.open(image_uri)
        png.load() # required for png.split()

        background = Image.new("RGB", png.size, (255, 255, 255))
        background.paste(png, mask=png.split()[3]) # 3 is the alpha channel

        base = os.path.split(image_uri)
        name = base[1].replace(".png",".jpg")

        background.save(base[0]+"/"+name, 'JPEG', quality=100)

        os.remove(image_uri)

In [24]:
for p in os.listdir(uri_to_path):
    for image in glob.glob(uri_to_path+"/"+p+"/*.png"):
        png_to_jpg(image)

In [9]:
#recorte e redimensionamento
#path, to_path, width, height, with_crop, with_resize
for path in os.listdir(uri_to_path):
    c_s1 = CropScalingImage(uri_to_path+"/"+path,uri_to_path+"/"+path, 528, True,True)
    c_s1.start()

**Gerar imagens**

*Executar uma das duas opões abaixo:*

In [6]:
uri_path = "C:/Users/Joelp/OneDrive/Imagens/tireoide/citologia"
uri_to_path = "C:/Users/Joelp/OneDrive/Imagens/tireoide/generated_cito"

for p in os.listdir(uri_path):

    for classe in os.listdir(uri_path+"/"+p):
        g = GenerateImage(uri_path+"/"+p+"/"+classe, uri_to_path+"/"+p+"/"+classe,20)
        g.start()




Foto 24-05-2021 20 00 48.jpg


Foto 24-05-2021 20 18 52.jpg


img1.jpg




______________________________________________________________

**Apagar imagens em excesso**

In [ ]:
classes = os.listdir(uri_)

for classe in classes:
    q = len(os.listdir("C:/Users/Joelp/OneDrive/Imagens/sementes/preparadas/21-04-2021/gerada/"+classe))
    qtd = qtd+q
    print(q)
    
print("Quantidade total:")
print(qtd)

In [ ]:
#
i = 1
images = os.listdir("imagens/sementes/deep_learning/milho")
for image in images:
    os.remove("imagens/sementes/deep_learning/milho/"+image)
    print(".", end="")
    if i == 1316:
        break
    i = i+1

**Converter imagens png para jpeg**

In [ ]:
list_classes = os.listdir(uri_path)

for classe in list_classes:
    images = os.listdir("sementes/deep_learning/"+classe)
    print(classe)
    for image in images:
        im = Image.open("sementes/deep_learning/"+classe+"/"+image)
        rgb_im = im.convert('RGB')
        rgb_im.save("sementes/deep_learning/"+classe+"/"+image)
        print(".", end="")

**Renomear as imagens**

In [ ]:
list_classes = os.listdir(uri_to_path)
for classe in list_classes:
    list_sementes = os.listdir(uri_to_path+"/"+classe)
    i = 0
    print("\n"+classe)
    for name in list_sementes:
        
        os.rename(uri_to_path+"/"+classe+"/"+name,uri_to_path+"/"+classe+"/"+classe+"_"+str(i)+".jpeg")
        i = i+1
        print(".",end="")